<a href="https://colab.research.google.com/github/baileysmoko/Fabric/blob/main/bitcoin_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()  # this will open a file picker


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Load BTC dataset ---
btc = pd.read_csv("New_Query (1).csv", parse_dates=["day"])
btc = btc.sort_values("day").reset_index(drop=True)

# --- Parameters ---
ema_short = 30
ema_long = 50
atr_period = 14
notional = 10000  # USD per trade

# --- Indicators ---
btc["EMA20"] = btc["close_usd"].ewm(span=ema_short, adjust=False).mean()
btc["EMA50"] = btc["close_usd"].ewm(span=ema_long, adjust=False).mean()

# --- ATR14 calculation (optional, for filtering if needed) ---
btc["close_diff"] = btc["close_usd"].diff().abs()
btc["ATR14"] = btc["close_diff"].rolling(atr_period).mean()

# --- Trading logic (long only, EMA crossover) ---
trades = []
in_position = False
entry_price = 0.0
entry_date = None

for i in range(1, len(btc)):
    row = btc.iloc[i]

    # Entry condition - long only
    if not in_position and row["EMA20"] > row["EMA50"]:
        in_position = True
        entry_price = row["close_usd"]
        entry_date = row["day"]

    # Exit condition - EMA crossover only
    elif in_position and row["EMA20"] < row["EMA50"]:
        exit_price = row["close_usd"]
        exit_date = row["day"]
        pnl = (exit_price - entry_price) / entry_price * notional
        trades.append({
            "entry_date": entry_date,
            "exit_date": exit_date,
            "year": exit_date.year,
            "position": "long",
            "entry_price": entry_price,
            "exit_price": exit_price,
            "pnl": pnl,
            "outcome": "win" if pnl > 0 else "loss"
        })
        in_position = False

# --- Create trades DataFrame ---
trades_df = pd.DataFrame(trades)
trades_df["cumulative_pnl"] = trades_df["pnl"].cumsum()

# --- Yearly Strategy PnL ---
yearly_strategy = trades_df.groupby("year")["pnl"].sum()

# --- Yearly Buy-and-Hold PnL ---
btc["year"] = btc["day"].dt.year
yearly_bh = {}

for year in sorted(btc["year"].unique()):
    year_data = btc[btc["year"] == year]
    if not year_data.empty:
        start_price = year_data["close_usd"].iloc[0]
        end_price = year_data["close_usd"].iloc[-1]
        bh_pnl = (end_price - start_price) / start_price * notional
        yearly_bh[year] = bh_pnl

yearly_bh = pd.Series(yearly_bh)

# --- Combine into Comparison Table ---
comparison = pd.DataFrame({
    "Strategy PnL": yearly_strategy,
    "Buy&Hold PnL": yearly_bh
}).fillna(0)

# --- Plot cumulative PnL ---
plt.figure(figsize=(12,6))
plt.plot(trades_df["exit_date"], trades_df["cumulative_pnl"], label="EMA20/50 Long-Only Strategy")
plt.title("BTC EMA20/50 Long-Only Backtest")
plt.xlabel("Date")
plt.ylabel("Cumulative PnL (USD)")
plt.legend()
plt.show()

# --- Output ---
print("\nAll Trades:")
print(trades_df[["entry_date","exit_date","year","position","entry_price","exit_price","pnl","outcome"]])

print("\nYearly PnL Comparison:")
print(comparison)

print("\nTotal Strategy PnL:", trades_df["pnl"].sum())
print("Total Buy&Hold PnL:", yearly_bh.sum())


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Load BTC dataset ---
btc = pd.read_csv("New_Query (1).csv", parse_dates=["day"])
btc = btc.sort_values("day").reset_index(drop=True)

# --- Notional ---
notional = 10000

# --- Compute daily returns ---
btc["daily_return"] = btc["close_usd"].pct_change().fillna(0)

# --- Compute Buy & Hold PnL ---
btc["bh_pnl"] = (btc["close_usd"] / btc["close_usd"].iloc[0] - 1) * notional

# --- Compute cumulative Buy & Hold ---
btc["cumulative_bh"] = btc["bh_pnl"]

# --- Sharpe ratio calculation ---
# Assuming risk-free rate = 0
daily_returns = btc["daily_return"]
sharpe_ratio_bh = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252)  # annualized

print(f"Buy & Hold Total PnL: {btc['bh_pnl'].iloc[-1]:.2f} USD")
print(f"Buy & Hold Annualized Sharpe Ratio: {sharpe_ratio_bh:.2f}")

# --- Optional: Plot Buy & Hold vs Strategy ---
plt.figure(figsize=(12,6))
plt.plot(btc["day"], btc["cumulative_bh"], label="Buy & Hold")
plt.title("Buy & Hold Strategy Performance")
plt.xlabel("Date")
plt.ylabel("Cumulative PnL (USD)")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import pandas as pd

def max_drawdown(cumulative):
    """Calculate max drawdown from cumulative pnl series."""
    roll_max = cumulative.cummax()
    drawdown = cumulative - roll_max
    return drawdown.min()

def calculate_metrics(subset, notional, use_daily_sharpe=False, btc=None):
    """Helper to calculate metrics for a subset of trades."""
    if subset.empty:
        return {
            "count": 0, "win_rate": 0, "avg_win": 0, "avg_loss": 0,
            "profit_factor": 0, "expectancy": 0, "total_pnl": 0, "sharpe": 0,
        }

    count = len(subset)
    wins = (subset["pnl"] > 0).sum()
    losses = (subset["pnl"] <= 0).sum()
    win_rate = wins / count if count > 0 else 0

    avg_win = subset.loc[subset["pnl"] > 0, "pnl"].mean()
    avg_loss = subset.loc[subset["pnl"] <= 0, "pnl"].mean()
    profit_factor = (
        subset.loc[subset["pnl"] > 0, "pnl"].sum() /
        abs(subset.loc[subset["pnl"] <= 0, "pnl"].sum())
        if losses > 0 else np.inf
    )

    expectancy = subset["pnl"].mean()
    total_pnl = subset["pnl"].sum()

    # Sharpe: per-trade Sharpe or daily Sharpe if BTC prices provided
    if use_daily_sharpe and btc is not None:
        daily_returns = btc["close_usd"].pct_change().fillna(0)
        sharpe = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252) if daily_returns.std() != 0 else 0
    else:
        trade_returns = subset["pnl"] / notional
        sharpe = trade_returns.mean() / trade_returns.std() if trade_returns.std() != 0 else 0

    return {
        "count": count,
        "win_rate": win_rate,
        "avg_win": avg_win,
        "avg_loss": avg_loss,
        "profit_factor": profit_factor,
        "expectancy": expectancy,
        "total_pnl": total_pnl,
        "sharpe": sharpe,
    }

if not trades_df.empty:
    # --- Overall metrics ---
    overall_metrics = calculate_metrics(trades_df, notional, use_daily_sharpe=False)

    equity_curve = trades_df["pnl"].cumsum()
    mdd = max_drawdown(equity_curve)

    # --- CAGR calculation based on total PnL and total days ---
    total_days = (btc["day"].iloc[-1] - btc["day"].iloc[0]).days
    years = total_days / 365.25
    cagr = ((overall_metrics["total_pnl"] / notional + 1) ** (1 / years) - 1) if years > 0 else 0

    print("\n--- Overall Strategy Performance ---")
    print(f"Total Trades: {overall_metrics['count']}")
    print(f"Win Rate: {overall_metrics['win_rate']:.2%}")
    print(f"Average Win: {overall_metrics['avg_win']:.2f} USD")
    print(f"Average Loss: {overall_metrics['avg_loss']:.2f} USD")
    print(f"Profit Factor: {overall_metrics['profit_factor']:.2f}")
    print(f"Expectancy (avg PnL per trade): {overall_metrics['expectancy']:.2f} USD")
    print(f"Total PnL: {overall_metrics['total_pnl']:.2f} USD")
    print(f"Sharpe Ratio (per-trade): {overall_metrics['sharpe']:.2f}")
    print(f"Max Drawdown: {mdd:.2f} USD")
    print(f"CAGR: {cagr:.2%}")

    # --- Yearly metrics ---
    yearly_metrics = {}
    print("\n--- Yearly Performance ---")
    for year, year_trades in trades_df.groupby("year"):
        yearly_metrics[year] = calculate_metrics(year_trades, notional)
        metrics = yearly_metrics[year]
        print(
            f"{year}: Trades = {metrics['count']}, "
            f"Win Rate = {metrics['win_rate']:.2%}, "
            f"PnL = {metrics['total_pnl']:.2f} USD, "
            f"Sharpe = {metrics['sharpe']:.2f}"
        )

else:
    print("No trades were generated.")


In [ ]:
import pandas as pd
import numpy as np
from itertools import product

def backtest_strategy(btc, ema_short, ema_long, notional=10000):
    btc = btc.copy()
    btc["EMA_short"] = btc["close_usd"].ewm(span=ema_short, adjust=False).mean()
    btc["EMA_long"] = btc["close_usd"].ewm(span=ema_long, adjust=False).mean()

    equity = [notional]  # start with initial capital
    in_position = False
    entry_price = 0.0
    trades = []  # list of trade PnLs

    for i in range(1, len(btc)):
        row = btc.iloc[i]

        if not in_position and row["EMA_short"] > row["EMA_long"]:
            # enter trade
            in_position = True
            entry_price = row["close_usd"]

        elif in_position and row["EMA_short"] < row["EMA_long"]:
            # exit trade
            exit_price = row["close_usd"]
            pnl = (exit_price - entry_price) / entry_price * notional
            trades.append(pnl)
            equity.append(equity[-1] + pnl)
            in_position = False
        else:
            # no trade
            equity.append(equity[-1])

    return pd.Series(equity, index=btc["day"][:len(equity)]), trades


def win_rate(trades):
    if len(trades) == 0:
        return 0.0
    return sum(1 for t in trades if t > 0) / len(trades)


def sharpe_ratio(equity, risk_free_rate=0.0, periods_per_year=252):
    """Compute annualized Sharpe ratio from equity curve."""
    # compute daily returns
    returns = equity.pct_change().dropna()

    # excess returns
    excess = returns - risk_free_rate / periods_per_year

    # annualized Sharpe
    return np.sqrt(periods_per_year) * excess.mean() / excess.std() if excess.std() != 0 else 0.0



def parameter_search(btc, notional=10000):
    ema_short_values = [10, 20, 30]
    ema_long_values = [50, 100, 150]

    results = []

    for ema_s, ema_l in product(ema_short_values, ema_long_values):
        if ema_s >= ema_l:
            continue

        equity, trades = backtest_strategy(btc, ema_s, ema_l, notional)

        results.append({
            "ema_short": ema_s,
            "ema_long": ema_l,
            "total_pnl": equity.iloc[-1] - notional,
            "win%": win_rate(trades),
            "sharpe": sharpe_ratio(equity),   # use equity curve now
            "num_trades": len(trades)
        })


    results_df = pd.DataFrame(results)
    return results_df


if __name__ == "__main__":
    btc = pd.read_csv("New_Query (1).csv", parse_dates=["day"])
    btc = btc.sort_values("day").reset_index(drop=True)

    results_df = parameter_search(btc)

    print("\n--- Parameter Search Results (Sorted by Sharpe) ---")
    print(results_df.sort_values("sharpe", ascending=False).head(10))

    print("\n--- Parameter Search Results (Sorted by Total PnL) ---")
    print(results_df.sort_values("total_pnl", ascending=False).head(10))


In [ ]:
import pandas as pd
import numpy as np

def backtest_strategy(btc, ema_short, ema_long, atr_period, atr_multiplier, notional=10000):
    """
    Runs EMA crossover backtest with ATR-based stop-loss.
    Returns list of trade PnLs.
    """
    btc = btc.copy()
    btc["EMA_short"] = btc["close_usd"].ewm(span=ema_short, adjust=False).mean()
    btc["EMA_long"] = btc["close_usd"].ewm(span=ema_long, adjust=False).mean()

    # Calculate ATR
    btc["close_diff"] = btc["close_usd"].diff().abs()
    btc["ATR"] = btc["close_diff"].rolling(atr_period).mean()

    trades = []
    in_position = False
    entry_price = 0.0
    stop_loss = 0.0

    for i in range(1, len(btc)):
        row = btc.iloc[i]
        atr_value = row["ATR"]

        # Entry condition
        if not in_position and row["EMA_short"] > row["EMA_long"]:
            in_position = True
            entry_price = row["close_usd"]
            stop_loss = entry_price - atr_multiplier * atr_value

        # Exit condition (EMA crossover or stop-loss hit)
        elif in_position:
            if row["EMA_short"] < row["EMA_long"] or row["close_usd"] <= stop_loss:
                exit_price = row["close_usd"]
                pnl = (exit_price - entry_price) / entry_price * notional
                trades.append(pnl)
                in_position = False

    return trades


def win_rate(trades):
    if len(trades) == 0:
        return 0.0
    return sum(1 for t in trades if t > 0) / len(trades)


def sharpe_ratio(trades):
    if len(trades) < 2:
        return 0.0
    returns = np.array(trades)
    return returns.mean() / returns.std() if returns.std() != 0 else 0.0


def atr_multiplier_search(btc, ema_short=20, ema_long=50, atr_period=14, notional=10000):
    """Search best ATR multiplier for Sharpe ratio."""
    atr_multipliers = np.arange(0.5, 5.5, 0.5)  # 0.5x to 5x ATR
    results = []

    for mult in atr_multipliers:
        trades = backtest_strategy(btc, ema_short, ema_long, atr_period, mult, notional)
        results.append({
            "atr_multiplier": mult,
            "total_pnl": np.sum(trades),
            "win%": win_rate(trades),
            "sharpe": sharpe_ratio(trades),
            "num_trades": len(trades)
        })

    results_df = pd.DataFrame(results)
    return results_df


if __name__ == "__main__":
    btc = pd.read_csv("New_Query (1).csv", parse_dates=["day"])
    btc = btc.sort_values("day").reset_index(drop=True)

    results_df = atr_multiplier_search(btc)

    print("\n--- ATR Multiplier Search Results (Sorted by Sharpe) ---")
    print(results_df.sort_values("sharpe", ascending=False).head(10))

    print("\n--- ATR Multiplier Search Results (Sorted by Total PnL) ---")
    print(results_df.sort_values("total_pnl", ascending=False).head(10))
